In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131352 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive  -o nonempty

In [0]:
LINE = 100000
START = '<s>'
STOP = '</s>'
DRIVE_DIR = 'drive/couplet/'
IN_NAME = DRIVE_DIR+'in.txt'
OUT_NAME = DRIVE_DIR+'out.txt'
DIC_NAME = DRIVE_DIR+'dictionary.dict'
EXTRA = '*'


def get_in(add_symbol=False):
    txt_in = []
    with open(IN_NAME,'r') as f:
        for i in range(LINE):
            ln = f.readline()[:-1]
            if add_symbol:
                ln += ' '+STOP
            txt_in.append(ln.split())
    return txt_in

def get_out(add_symbol=False):
    txt_out = []
    with open(OUT_NAME,'r') as f:
        for i in range(LINE):
            ln = f.readline()[:-1]
            if add_symbol:
                ln = START+' '+ln+' '+STOP
            txt_out.append(ln.split())
            # txt_out.append(ln.split())
    return txt_out

In [2]:
import numpy as np
import keras.backend as K
import gc
import sys
from keras.layers import Dense,LSTM,Embedding,Input,GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from gensim.corpora import Dictionary
gc.collect()
K.clear_session()

Using TensorFlow backend.


In [0]:
BATCH = 50
EPOCHS = 100
LR = 1e-4
DECAY = 1e-6
MODEL_NAME = DRIVE_DIR+'result/model-190215.h5'

BATCH_NUM = int(np.ceil(float(LINE)/BATCH))

dic = Dictionary.load(DIC_NAME)

txt_in = get_in(True)
txt_out = get_out(True)

def get_max_len(txt):
    return max(list(map(lambda ln: len(ln),txt)))

MAX_INPUT_LEN = get_max_len(txt_in)
MAX_TARGET_LEN = get_max_len(txt_out)
VOCAB_NUM = len(dic.token2id.values())

LSTM_CELLS = 256
EBD_CELLS = 200

In [0]:
def get_seq(dic,tin,max_input_len,start,stop,extra):
    txt_in = tin.strip()
    seq = np.zeros([1,max_input_len])
    words = dic.token2id.keys()
    txt_len = len(txt_in)
    for i in range(txt_len):
        if txt_in[i] in words:
            seq[0,i] = dic.token2id[txt_in[i]]
            print(txt_in[i],'\t',dic.token2id[txt_in[i]])
        else:
            seq[0,i] = dic.token2id[extra]
    seq[0,txt_len] = dic.token2id[stop]
    print(seq)
    return seq

In [0]:
def gen_seq(dic,tin,tout,batch,batch_num,max_input_len,max_target_len):
    token2id = dic.token2id
    vocab_num = len(token2id.values())
    while True:
        for i in range(batch_num):
            e_in = np.zeros([batch,max_input_len])
            d_in = np.zeros([batch,max_target_len])
            d_out = np.zeros([batch,max_target_len,vocab_num])
            tin_b = tin[i*batch:(i+1)*batch]
            tout_b = tout[i*batch:(i+1)*batch]
            for j in range(batch):
                for k,w in enumerate(tin_b[j]):
                    if w in token2id.keys():
                        e_in[j,k] = token2id[w]
                    else:
                        e_in[i,k] = token2id[EXTRA]
                for k,w in enumerate(tout_b[j]):
                    if w in token2id.keys():
                        d_in[j,k] = token2id[w]
                        if k > 0:
                            d_out[j,k-1,token2id[w]] = 1
                    else:
                        d_in[j,k] = token2id[EXTRA]
                        if k > 0:
                            d_out[j,k-1,token2id[EXTRA]] = 1
            yield [e_in,d_in],d_out

In [6]:
e_ipt = Input(shape=[MAX_INPUT_LEN])
e_ebd = Embedding(input_dim=VOCAB_NUM,output_dim=EBD_CELLS)
e_lstm = LSTM(LSTM_CELLS,return_state=True)

e_ebd_o = e_ebd(e_ipt)
e_o = e_lstm(e_ebd_o)

d_ipt = Input(shape=[MAX_TARGET_LEN])
d_ebd = Embedding(input_dim=VOCAB_NUM,output_dim=EBD_CELLS)
d_lstm = LSTM(LSTM_CELLS,return_sequences=True,return_state=True)
d_fc = Dense(VOCAB_NUM,activation='softmax')

d_ebd_o = d_ebd(d_ipt)
d_lstm_o = d_lstm(d_ebd_o,initial_state=e_o[1:])
d_fc_o = d_fc(d_lstm_o[0])

model = Model([e_ipt,d_ipt],d_fc_o)
chpt = ModelCheckpoint(MODEL_NAME,save_weights_only=True)
adam = Adam(lr=LR,decay=DECAY)
model.compile(optimizer=adam,loss='categorical_crossentropy')

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
model.load_weights(MODEL_NAME)
model.fit_generator(gen_seq(
    dic,
    txt_in,
    txt_out,
    BATCH,
    BATCH_NUM,
    MAX_INPUT_LEN,
    MAX_TARGET_LEN
),steps_per_epoch=BATCH_NUM,epochs=EPOCHS,callbacks=[chpt])

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/100
2000/2000 [==============================] - 422s 211ms/step - loss: 0.9373
Epoch 2/100
2000/2000 [==============================] - 420s 210ms/step - loss: 0.9348
Epoch 3/100
2000/2000 [==============================] - 419s 210ms/step - loss: 0.9333
Epoch 4/100
2000/2000 [==============================] - 419s 210ms/step - loss: 0.9309
Epoch 5/100
2000/2000 [==============================] - 420s 210ms/step - loss: 0.9287
Epoch 6/100
2000/2000 [==============================] - 420s 210ms/step - loss: 0.9265
Epoch 7/100
2000/2000 [==============================] - 419s 210ms/step - loss: 0.9247
Epoch 8/100
2000/2000 [==============================] - 419s 210ms/step - loss: 0.9225
Epoch 9/100
2000/2000 [==============================] - 419s 209ms/step - loss: 0.9200
Epoch 10/100
2000/2000 [==============================] - 419s 210ms/step - loss: 